In [ ]:
import numpy as np

from sklearn.model_selection import train_test_split
from keras.utils import to_categorical

from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

In [ ]:
sign_labels = []
with open('./sign_labels.csv', 'r') as file:
    sign_labels = file.read().splitlines()
file.close()

sign_labels = np.array(sign_labels)
        

In [ ]:
class TrainModel:
    def __init__(self, data_set_path, model_save_path) -> None:
        self.data_set_path = data_set_path
        self.model_save_path = model_save_path
        self.model = Sequential([
            LSTM(64, return_sequences=True, activation='relu', input_shape=(21 * 2, )), # TODO: may wanna check this
            LSTM(128, return_sequences=True, activation='relu'),
            LSTM(64, return_sequences=False, activation='relu'),
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(sign_labels.shape[0], activation='softmax'), # could also be the number of unique labels
        ])
    
    def load_data_set(self):
        X_data = np.loadtxt(self.data_set_path, delimiter=',', dtype='float32', usecols=list(range(1, (21 * 2) + 1)))
        y_data = np.loadtxt(self.data_set_path, delimiter=',', dtype='int32', usecols=(0))
        
        return train_test_split(X_data, y_data, test_size=0.2, random_state=42)
    
    def save_model(self):
        self.model.save(self.model_save_path)

In [ ]:
data_set_path = './data_set.csv'
model_save_path = './models/model.h5'

Model = TrainModel(data_set_path, model_save_path)

X_train, X_test, y_train, y_test = Model.load_data_set()

In [ ]:
Model.model.summary()

In [ ]:
checkpoint = ModelCheckpoint(Model.model_save_path, verbose=1, save_weights_only=False) # what is this ?
early_stopping = EarlyStopping(patience=20, verbose=1) # what is this ?
tensor_board = TensorBoard(log_dir='./logs', histogram_freq=1)

In [ ]:
Model.model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy', # diff between this and categorical_crossentropy
    metrics=['accuracy'] # diff between this and categorical_accuracy
)

In [ ]:
Model.model.fit(
    X_train,
    y_train,
    epochs=1000,
    batch_size=64,
    validation_data=(X_test, y_test),
    callbacks=[tensor_board, checkpoint, early_stopping]
)

In [ ]:
val_loss, val_acc = Model.model.evaluate(X_test, y_test, batch_size=64)

In [ ]:
Model.save_model()